In [9]:
# Generic Earth Engine Exporter (GRID or POINTS from CSV)

# Requirements: Google Colab + Earth Engine enabled
# - Mounts Google Drive and writes CSVs there
# - Datasets, dates, bands, mode
# - MODE = "GRID"   => sample all pixels over regions (e.g., states) at each dataset's native resolution
# - MODE = "POINTS" => sample at coordinates from a CSV (lon, lat[, id]); user SCALE_M is ignored

import time
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import ee
from google.colab import drive

# 0) Google Drive + EE init
drive.mount('/content/drive')
DRIVE_FOLDER = "GEE_Exports_Generic"  # change as needed
drive_path = Path('/content/drive/My Drive/')

# Authenticate & initialize EE
ee.Authenticate()  # In Colab, follow the popup flow
ee.Initialize(project='ee-your_project_id')  # <-- your EE project id

# 1) Configuration

# MODE: "GRID" or "POINTS"
MODE = "GRID"

# Dates - using inclusive start, exclusive end
START_DATE = ''
END_DATE   = ''

# Sampling resolution (meters)
# NOTE: Ignored in POINTS mode. In GRID mode we ALWAYS use each dataset's native/default scale.
SCALE_M = 1000  # kept for backward compatibility; not used anymore

# Dataset(s)
DATASETS = {
    # 1) DAYMET daily (~1 km)
    "DAYMET_V4": {
        "collection_id": "NASA/ORNL/DAYMET_V4",
        "default_bands": ['prcp', 'tmin', 'tmax', 'vp', 'srad', 'dayl'],
        "note": "Daily Daymet; USA & parts of Canada/Mexico",
        "default_scale": 1000  # native ~1 km
    },
    # 2) ERA5-Land daily aggregated
    "ERA5_LAND_DAILY": {
        "collection_id": "ECMWF/ERA5_LAND/DAILY_AGGR",
        "default_bands": [
            'total_precipitation_sum',
            'temperature_2m',
        ],
        "note": "Daily aggregated ERA5-Land; global coverage",
        "default_scale": 11132  # ~0.1° ~= 11.1 km at equator, used consistently
    }
}

# Choose which datasets and which bands from each to export.
SELECTIONS = [
    {"name": "DAYMET_V4", "bands": ['prcp', 'tmin', 'tmax']},
    {"name": "ERA5_LAND_DAILY", "bands": ['total_precipitation_sum', 'temperature_2m']},
]

# GRID mode region config
USE_CONUS_STATES = True
STATE_EXCLUDE = ['']               # exclude list if needed
ONLY_STATES = ['']        # subset if desired

# POINTS mode CSV config
POINTS_CSV = "/content/drive/My Drive/SMAP_locations/coord_df.csv"  # columns: lon, lat [, location_id]
POINTS_LON_COL = "lon"
POINTS_LAT_COL = "lat"
POINTS_ID_COL  = "location_id"   # optional; auto-generated if missing

# Export knobs
EXPORT_FOLDER = DRIVE_FOLDER
PER_STATE_FILE_IN_GRID_MODE = True  # one file per region (True) vs one global file (False)
CLIENT_SIDE_CHUNK_SLEEP = 1         # seconds between starting export tasks

# 2) Helpers

def get_image_collection(ds_name, start_date, end_date, selected_bands=None):
    """Return ee.ImageCollection filtered by date and selected bands."""
    meta = DATASETS[ds_name]
    col = ee.ImageCollection(meta["collection_id"]).filterDate(start_date, end_date)
    bands = selected_bands if selected_bands else meta["default_bands"]
    col = col.select(bands)
    return col, bands, meta

def add_lonlat(feature):
    """Add lon/lat properties from geometry."""
    coords = ee.Feature(feature).geometry().coordinates()
    return feature.set({'lon': coords.get(0), 'lat': coords.get(1)})

def sample_ic_over_region_grid(ic, region_geom, scale_m):
    """Sample every pixel center at given scale over region for every image in collection."""
    def sample_one_image(img):
        date_str = img.date().format('YYYY-MM-dd')
        fc = (
            img.sample(region=region_geom,
                       scale=scale_m,
                       geometries=True)
              .map(lambda f: f.set("date", date_str))
              .map(add_lonlat)
        )
        return fc
    return ic.map(sample_one_image).flatten()

def sample_ic_at_points(ic, points_fc, scale_m):
    """Sample an image collection at given point features (lon/lat)."""
    def sample_one_image(img):
        date_str = img.date().format('YYYY-MM-dd')
        fc = (
            img.sampleRegions(collection=points_fc,
                              scale=scale_m,
                              geometries=True)
              .map(lambda f: f.set("date", date_str))
              .map(add_lonlat)
        )
        return fc
    return ic.map(sample_one_image).flatten()

def export_fc_to_drive(fc, description, file_prefix, folder):
    """Start a Drive export task for the given FeatureCollection."""
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        folder=folder,
        description=description,
        fileNamePrefix=file_prefix,
        fileFormat='CSV'
    )
    task.start()
    print(f"  ✅ Export started: {description}  ->  Drive/{folder}/{file_prefix}*.csv")

def build_points_fc_from_csv(path, lon_col="lon", lat_col="lat", id_col=None):
    """Read CSV and build ee.FeatureCollection of point features."""
    df = pd.read_csv(path)
    if lon_col not in df.columns or lat_col not in df.columns:
        raise ValueError(f"CSV must contain '{lon_col}' and '{lat_col}' columns.")
    if id_col and id_col not in df.columns:
        print(f"  (Info) '{id_col}' not found, will auto-generate IDs.")

    feats = []
    for i, row in df.iterrows():
        lon = float(row[lon_col])
        lat = float(row[lat_col])
        props = {}
        if id_col and id_col in df.columns:
            # keep as string but safe
            props["site_id"] = str(row[id_col]).strip()
        else:
            props["site_id"] = f"pt_{i}"
        feat = ee.Feature(ee.Geometry.Point([lon, lat]), props)
        feats.append(feat)
    return ee.FeatureCollection(feats)

def get_conus_states_fc(only_states=None, exclude_states=None):
    """Return FeatureCollection of CONUS states (optionally filtered)."""
    states = ee.FeatureCollection("TIGER/2018/States")
    if exclude_states:
        states = states.filter(ee.Filter.inList("NAME", exclude_states).Not())
    if only_states:
        states = states.filter(ee.Filter.inList("NAME", only_states))
    return states

# 3) Main logic per MODE

print(f"\nMode: {MODE}")
print(f"Date range: [{START_DATE}, {END_DATE})")
print(f"Export folder: {EXPORT_FOLDER}")
if MODE.upper() == "POINTS":
    print("Scale: Ignored in POINTS mode (using dataset-native scale internally).")
else:
    print("Scale: Using dataset-native/default scale (original resolution).")

if MODE.upper() == "GRID":
    # Build region FeatureCollection
    if USE_CONUS_STATES:
        regions_fc = get_conus_states_fc(
            only_states=ONLY_STATES if ONLY_STATES else None,
            exclude_states=STATE_EXCLUDE
        )
        region_list = regions_fc.aggregate_array("NAME").getInfo()
        print(f"Regions (states) to process: {region_list}")
    else:
        raise ValueError("Set USE_CONUS_STATES=False only if you provide your own regions FC.")

    # Loop datasets (always at native/default resolution)
    for sel in SELECTIONS:
        ds_name = sel["name"]
        bands = sel.get("bands")
        ic, used_bands, meta = get_image_collection(ds_name, START_DATE, END_DATE, bands)
        scale_for_ds = meta.get("default_scale", 1000)  # ORIGINAL RESOLUTION

        print(f"\n=== DATASET: {ds_name} ({meta['collection_id']}) ===")
        print(f"Bands: {used_bands}")
        print(f"Sampling scale (native/default, m): {scale_for_ds}")

        if PER_STATE_FILE_IN_GRID_MODE:
            for st_name in region_list:
                print(f"\n-- Region: {st_name} --")
                geom = regions_fc.filter(ee.Filter.eq("NAME", st_name)).geometry()
                samples = sample_ic_over_region_grid(ic, geom, scale_for_ds)

                safe_st = st_name.replace(' ', '_').lower()
                file_prefix = f"{ds_name.lower()}_{safe_st}_{START_DATE.replace('-','')}_{END_DATE.replace('-','')}"
                desc = f"{ds_name}_{safe_st}_{START_DATE}_to_{END_DATE}_grid"
                export_fc_to_drive(samples, desc, file_prefix, EXPORT_FOLDER)
                time.sleep(CLIENT_SIDE_CHUNK_SLEEP)
        else:
            print("\n-- Merging all regions into a single export --")
            merged = ee.FeatureCollection([])
            def per_region(st_name):
                geom = regions_fc.filter(ee.Filter.eq("NAME", st_name)).geometry()
                fc = sample_ic_over_region_grid(ic, geom, scale_for_ds)
                return fc.map(lambda f: f.set("region_name", st_name))

            for st_name in region_list:
                merged = merged.merge(per_region(st_name))

            file_prefix = f"{ds_name.lower()}_regions_{START_DATE.replace('-','')}_{END_DATE.replace('-','')}"
            desc = f"{ds_name}_regions_{START_DATE}_to_{END_DATE}_grid"
            export_fc_to_drive(merged, desc, file_prefix, EXPORT_FOLDER)
            time.sleep(CLIENT_SIDE_CHUNK_SLEEP)

elif MODE.upper() == "POINTS":
    # Build points FC from CSV
    print(f"Reading points CSV: {POINTS_CSV}")
    points_fc = build_points_fc_from_csv(
        POINTS_CSV, lon_col=POINTS_LON_COL, lat_col=POINTS_LAT_COL, id_col=POINTS_ID_COL
    )
    npts = points_fc.size().getInfo()
    print(f"Points loaded: {npts}")

    # Loop datasets (SCALE_M IGNORED; use native/default)
    for sel in SELECTIONS:
        ds_name = sel["name"]
        bands = sel.get("bands")
        ic, used_bands, meta = get_image_collection(ds_name, START_DATE, END_DATE, bands)
        scale_for_ds = meta.get("default_scale", 1000)  # use dataset-native/default

        print(f"\n=== DATASET: {ds_name} ({meta['collection_id']}) ===")
        print(f"Bands: {used_bands}")
        print(f"Sampling scale in POINTS mode (native/default, m): {scale_for_ds} (user SCALE_M ignored)")

        samples = sample_ic_at_points(ic, points_fc, scale_for_ds)

        file_prefix = f"{ds_name.lower()}_points_{START_DATE.replace('-','')}_{END_DATE.replace('-','')}"
        desc = f"{ds_name}_points_{START_DATE}_to_{END_DATE}"
        export_fc_to_drive(samples, desc, file_prefix, EXPORT_FOLDER)
        time.sleep(CLIENT_SIDE_CHUNK_SLEEP)

else:
    raise ValueError("MODE must be 'GRID' or 'POINTS'.")

print("\nAll export tasks have been submitted.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Mode: GRID
Date range: [2018-01-01, 2018-07-16)
Export folder: GEE_Exports_Generic
Scale: Using dataset-native/default scale (original resolution).
Regions (states) to process: ['Arizona']

=== DATASET: DAYMET_V4 (NASA/ORNL/DAYMET_V4) ===
Bands: ['prcp', 'tmin', 'tmax']
Sampling scale (native/default, m): 1000

-- Region: Arizona --
  ✅ Export started: DAYMET_V4_arizona_2018-01-01_to_2018-07-16_grid  ->  Drive/GEE_Exports_Generic/daymet_v4_arizona_20180101_20180716*.csv

=== DATASET: ERA5_LAND_DAILY (ECMWF/ERA5_LAND/DAILY_AGGR) ===
Bands: ['total_precipitation_sum', 'temperature_2m']
Sampling scale (native/default, m): 11132

-- Region: Arizona --
  ✅ Export started: ERA5_LAND_DAILY_arizona_2018-01-01_to_2018-07-16_grid  ->  Drive/GEE_Exports_Generic/era5_land_daily_arizona_20180101_20180716*.csv

All export tasks have been submitted.
